In [1]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
root_dir = "D:/UoA_Research/data_7"
intr_param = {}

col = 7
row = 11
square_size = 43.
criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 30, 0.001)
objp = np.zeros((col*row, 3), np.float32)
objp[:, :2] = np.mgrid[0:col, 0:row].T.reshape(-1, 2) * square_size

In [2]:
# (fx, fy, cx, cy, h, w, k1, k2, p1, p2, k3, k4, k5, k6)
# intrinsic parameters for azure kinect
# intr_param['azure_kinect_0_depth'] = (504.615, 504.687, 322.52, 321.885, 576, 640, 0.549511, -0.000578435, -0.000103262, 2.06755e-05, -0.00113635, 0.892186, 0.107761, -0.00680388)
# intr_param['azure_kinect_1_depth'] = (504.615, 504.687, 322.52, 321.885, 576, 640, 0.549511, -0.000578435, -0.000103262, 2.06755e-05, -0.00113635, 0.892186, 0.107761, -0.00680388)
# intr_param['azure_kinect_2_depth'] = (504.615, 504.687, 322.52, 321.885, 576, 640, 0.549511, -0.000578435, -0.000103262, 2.06755e-05, -0.00113635, 0.892186, 0.107761, -0.00680388)
#
# intr_param['azure_kinect_0_color'] = (980.086, 980.12, 1019.92, 783.953, 1536, 2048, 0.569016, -2.70663, 0.000215031, -0.000383734, 1.51867, 0.447548, -2.5313, 1.44786)
# intr_param['azure_kinect_1_color'] = (980.086, 980.12, 1019.92, 783.953, 1536, 2048, 0.569016, -2.70663, 0.000215031, -0.000383734, 1.51867, 0.447548, -2.5313, 1.44786)
# intr_param['azure_kinect_2_color'] = (980.086, 980.12, 1019.92, 783.953, 1536, 2048, 0.569016, -2.70663, 0.000215031, -0.000383734, 1.51867, 0.447548, -2.5313, 1.44786)

# intr_param['kinect_v2_depth'] = (366.193, 366.193, 207.085, 256.684, 512, 424)
# intr_param['kinect_v2_color'] = (1081.37, 1081.37, 539.5, 959.5, 1920, 1080)

In [2]:
cam_list = ['azure_kinect_0', 'azure_kinect_1', 'azure_kinect_2', 'kinect_v2_1', 'kinect_v2_2', 'event_camera', 'polar']
start_idx = [0, 0, 0, 0, 0, 0, 0]
n_list = [60, 60, 60, 40, 40, 40, 40]
# cam_list = ['azure_kinect_0', 'azure_kinect_1', 'azure_kinect_2']
# start_idx = [0, 0, 0]
# n_list = [60, 60, 60]

for idx, cam in enumerate(cam_list):
    dir = "%s/%s_calib_snap" % (root_dir, cam)
    n = n_list[idx]  # number of images
    
    # color image
    h, w = 0, 0
    obj_points = []
    img_points = []
    for i in range(n):
        if 'polar' in cam:
            filename = '%s/polar0_%04i.jpg' % (dir, i)
            img = cv2.imread(filename)
            gray = img[:, :, 0]
        elif 'event' in cam:
            filename = '%s/fullpic%04i.jpg' % (dir, i)
            img = cv2.imread(filename)
            gray = img[:, :, 0]
        else:
            filename = '%s/color%04i.jpg' % (dir, i)
            img = cv2.imread(filename)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        h, w = img.shape[0], img.shape[1]
        ret, corners = cv2.findChessboardCorners(gray, (col, row), None)
        print(ret, filename)
        if ret:
            obj_points.append(objp)
            corners2 = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), criteria)
            img_points.append(corners2)
            cv2.drawChessboardCorners(img, (col, row), corners2, ret)
            cv2.imshow('img', cv2.resize(img, (int(img.shape[1]/2), int(img.shape[0]/2))))
            cv2.waitKey(50)
    
    cv2.destroyAllWindows()
    ret, mtx, dist, rvecs, tvecs = \
        cv2.calibrateCamera(obj_points, img_points, (w, h), None, None, flags=cv2.CALIB_RATIONAL_MODEL)
    print("ret:", ret)
    print("mtx:\n", mtx)
    print("dist:\n", dist)  # distortion cofficients = (k1,k2,p1,p2,k3,k4,k5,k6)
    
    # (fx, fy, cx, cy, h, w)
    params = (mtx[0, 0], mtx[1, 1], mtx[0, 2], mtx[1, 2], h, w) + tuple(dist[0, 0:8])
    print(params)
    if 'kinect' in cam:
        intr_param['%s_color' % cam] = params
    else:
        intr_param['%s' % cam] = params
    
    if 'kinect' in cam:
        # depth image
        h, w = 0, 0
        obj_points = []
        img_points = []
        for i in range(n):
            filename = '%s/infrared%04i.png' % (dir, i)
            img = cv2.imread(filename, -1)
            h, w = img.shape[0], img.shape[1]
            for thr in [0.02, 0.05, 0.1]:
                gray = np.clip(img.astype(np.float32) * thr, 0, 255).astype('uint8')
                ret, corners = cv2.findChessboardCorners(gray, (col, row), None)
                if ret:
                    break
            print(ret, filename)
            if ret:
                obj_points.append(objp)
                corners2 = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), criteria)
                img_points.append(corners2)
                cv2.drawChessboardCorners(gray, (col, row), corners2, ret)
                cv2.imshow('img', gray)
                cv2.waitKey(50)
        
        cv2.destroyAllWindows()
        ret, mtx, dist, rvecs, tvecs = \
            cv2.calibrateCamera(obj_points, img_points, (w, h), None, None, flags=cv2.CALIB_RATIONAL_MODEL)
        print("ret:", ret)
        print("mtx:\n", mtx)
        print("dist:\n", dist)  # distortion cofficients = (k1,k2,p1,p2,k3,k4,k5,k6)
        
        # (fx, fy, cx, cy, h, w)
        params = (mtx[0, 0], mtx[1, 1], mtx[0, 2], mtx[1, 2], h, w) + tuple(dist[0, 0:8])
        print(params)
        intr_param['%s_depth' % cam] = params


True D:/UoA_Research/data_7/azure_kinect_0_calib_snap/color0000.jpg
True D:/UoA_Research/data_7/azure_kinect_0_calib_snap/color0001.jpg
True D:/UoA_Research/data_7/azure_kinect_0_calib_snap/color0002.jpg
True D:/UoA_Research/data_7/azure_kinect_0_calib_snap/color0003.jpg
True D:/UoA_Research/data_7/azure_kinect_0_calib_snap/color0004.jpg
True D:/UoA_Research/data_7/azure_kinect_0_calib_snap/color0005.jpg
True D:/UoA_Research/data_7/azure_kinect_0_calib_snap/color0006.jpg
True D:/UoA_Research/data_7/azure_kinect_0_calib_snap/color0007.jpg
True D:/UoA_Research/data_7/azure_kinect_0_calib_snap/color0008.jpg
True D:/UoA_Research/data_7/azure_kinect_0_calib_snap/color0009.jpg
True D:/UoA_Research/data_7/azure_kinect_0_calib_snap/color0010.jpg
True D:/UoA_Research/data_7/azure_kinect_0_calib_snap/color0011.jpg
True D:/UoA_Research/data_7/azure_kinect_0_calib_snap/color0012.jpg
True D:/UoA_Research/data_7/azure_kinect_0_calib_snap/color0013.jpg
True D:/UoA_Research/data_7/azure_kinect_0_calib

In [3]:
# save
for k, v in intr_param.items():
    print(k, v)
with open('%s/intrinsic_param.pkl' % root_dir, 'wb') as f:
    pickle.dump(intr_param, f)



azure_kinect_0_color (976.7838346046938, 976.6954700661433, 1000.7015978283123, 784.6995695911885, 1536, 2048, 13.366022046162477, -11.15344330351859, 0.00016147564080253588, -0.008163149588165207, -88.58085389918612, 13.318338772065024, -12.813268592939618, -83.7145525534381)
azure_kinect_0_depth (503.0663909874723, 503.2110805572531, 311.33891831250764, 321.9362422108562, 576, 640, -28.34559721362226, 405.10560089798076, -0.00018551407336784846, 0.001373663043499764, 75.66967336693958, -27.991398216341494, 395.1877482416642, 215.6202849517827)
azure_kinect_1_color (972.7423807781718, 972.3850924769915, 1001.0666252429908, 784.8871379179792, 1536, 2048, -23.06510632923673, 223.65759622734413, 0.001576580975294305, -0.00849949885809845, 373.3356298674388, -23.140398630220908, 225.67456465888569, 351.94902429601433)
azure_kinect_1_depth (502.43212671422515, 502.70714123230726, 310.18844369185683, 337.1785766970918, 576, 640, 11.419185118646586, 120.61415817478067, -0.0001611932127833309

In [1]:
import cv2
import numpy as np

dir = r"C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration"
cam_list = ['azure_kinect_0', 'azure_kinect_1', 'azure_kinect_2', 'kinect_v2_1', 'kinect_v2_2']
start_idx = [20, 20, 0, 100, 0]
cam_R, cam_T, cam_M = [], [], []
n_list = [80, 100, 100, 20, 20]
col = 8
row = 11
square_size = 43.
objp = np.zeros((col*row, 3), np.float32)
objp[:, :2] = np.mgrid[0:col, 0:row].T.reshape(-1, 2) * square_size
h = 0
w = 0
corner_m = []
obj_m = []
for idx, cam in enumerate(cam_list):
    obj_points = []
    img_points = []
    cam_dir = "%s/%s_calib_snap" % (dir, cam)
    for i in range(n_list[idx]):
        filename = '%s/color%04i.jpg' % (cam_dir, i + start_idx[idx])
        print(i)
        img = cv2.imread(filename)
        h, w = img.shape[0], img.shape[1]
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (col, row), None)
        if ret:
            print(filename)
            corners2 = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), None)
            obj_points.append(objp)
            img_points.append(np.squeeze(corners2))
    
    obj_points= np.array(obj_points).astype('float32')
    img_points=np.array(img_points).astype('float32')
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, (w, h), None, None, flags=cv2.CALIB_RATIONAL_MODEL)   
    retval, rvec, tvec = cv2.solvePnP(objectPoints=obj_points[0], imagePoints=img_points[0], cameraMatrix=mtx, distCoeffs=dist)
    cam_R.append(rvec)
    cam_T.append(tvec)


0
C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration/azure_kinect_0_calib_snap/color0020.jpg
1
C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration/azure_kinect_0_calib_snap/color0021.jpg
2
C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration/azure_kinect_0_calib_snap/color0022.jpg
3
C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration/azure_kinect_0_calib_snap/color0023.jpg
4
C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration/azure_kinect_0_calib_snap/color0024.jpg
5
C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration/azure_kinect_0_calib_snap/color0025.jpg
6
C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration/azure_kinect_0_calib_snap/color0026.jpg
7
C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration/azure_kinect_0_calib_snap/color0027.jpg
8
C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration/azure_kinect_0

In [4]:
RT = {}
for i in range(len(cam_list)):
    if i < len(cam_list) - 1: 
        r1 = cv2.Rodrigues(cam_R[i])[0]
        r2 = cv2.Rodrigues(cam_R[i + 1])[0]
        t1 = cam_T[i]
        t2 = cam_T[i + 1]
        r12 = np.dot(r1, r2.T)
        t12 = t1 - np.dot(r12, t2)
        RT[cam_list[i] + ' ' + cam_list[i+1] + '_R'] = r12
        RT[cam_list[i] + ' ' + cam_list[i+1] + '_T'] = t12
    else:
        r1 = cv2.Rodrigues(cam_R[i])[0]
        r2 = cv2.Rodrigues(cam_R[0])[0]
        t1 = cam_T[i]
        t2 = cam_T[0]
        r12 = np.dot(r1, r2.T)
        t12 = t1 - np.dot(r12, t2)
        RT[cam_list[i] + ' ' + cam_list[0] + '_R'] = r12
        RT[cam_list[i] + ' ' + cam_list[0] + '_T'] = t12

In [6]:
for key, value in RT.items():
    print (key)
    print(value)

azure_kinect_0 azure_kinect_1_R : [[ 0.92609197  0.24322192  0.28843847]
 [-0.13833465  0.93013319 -0.3401702 ]
 [-0.35102305  0.27512786  0.8950349 ]]
azure_kinect_0 azure_kinect_1_T : [[-617.81594528]
 [ 511.12514435]
 [ 293.21624109]]
azure_kinect_1 azure_kinect_2_R : [[ 0.90116792 -0.13217331 -0.41282756]
 [ 0.25160688  0.93501647  0.24987632]
 [ 0.35297359 -0.32905078  0.87586256]]
azure_kinect_1 azure_kinect_2_T : [[ 667.31188942]
 [-304.21121464]
 [ 240.70810311]]
azure_kinect_2 kinect_v2_1_R : [[ 0.08880723  0.94130817 -0.32565657]
 [-0.85481525 -0.09579799 -0.51001336]
 [-0.51127699  0.32366907  0.79613703]]
azure_kinect_2 kinect_v2_1_T : [[369.39152165]
 [643.42127741]
 [ 64.86744636]]
kinect_v2_1 kinect_v2_2_R : [[-0.30169895 -0.84429702 -0.44287728]
 [ 0.73367112  0.09105257 -0.67337665]
 [ 0.60885502 -0.5280833   0.59196587]]
kinect_v2_1 kinect_v2_2_T : [[ 498.55068021]
 [1192.2347985 ]
 [ 822.77487114]]
kinect_v2_2 azure_kinect_0_R : [[ 0.40277319 -0.13526016  0.90525049]